# <div style="font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:200%; text-align:center;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0">Santa 2023 - NxNxN Cube Monte Carlo</div>
#### <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:150%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" >TABLE OF CONTENTS<br><div>
* [IMPORTS](#1)
* [LOAD DATA](#2)
* [FUNCTIONS](#3)
* [SOLVE](#4)

Using repo from dwalton76: https://github.com/dwalton76/rubiks-cube-NxNxN-solver/tree/master

I want to explore solving the cube after manipulating the initial state of N-labled soltuions. Modifying the initial 
state may generate a path to the solution

<a id="1"></a>
# <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:120%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" > IMPORTS<br><div> 

In [1]:
import zipfile
import sqlite3
import random
import pandas as pd
import numpy as np

import os

os.chdir('rubiks-cube-NxNxN-solver')
# Print the current working directory
print("Current Working Directory: ", os.getcwd())

database_file = '../../solutions.db'
solution_method = "dwalton76 NxNxN algo"

Current Working Directory:  /Users/seanbearden/PycharmProjects/Kaggle/src/santa-2023/magic-cube/rubiks-cube-NxNxN-solver


<a id="2"></a>
# <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:120%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" >LOAD DATA<br><div> 

In [2]:
with zipfile.ZipFile('../../../../res/data/santa-2023.zip', 'r') as z:
    
    with z.open('puzzle_info.csv') as f:
        puzzle_info = pd.read_csv(f, index_col = 'puzzle_type')        
                
    with z.open('puzzles.csv') as f:
        puzzles = pd.read_csv(f, index_col='id')
    
    with z.open('sample_submission.csv') as f:
        submission = pd.read_csv(f)

<a id="3"></a>
# <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:120%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" >FUNCTIONS<br><div>
 

In [3]:
# U = ['U', 'F', 'R', 'B', 'L', 'D']

def state2ubl(state):
    U_dict = {
        'A': 'U',
        'B': 'F',
        'C': 'R',
        'D': 'B',
        'E': 'L',
        'F': 'D',
    }
    state_split = state.split(';')
    dim = int(np.sqrt(len(state_split) // 6))
    dim_2 = dim**2
    
    s = ''.join([U_dict[f] for f in state_split])
    
    return s[:dim_2] + s[2*dim_2:3*dim_2] + s[dim_2:2*dim_2] + s[5*dim_2:] + s[4*dim_2:5*dim_2] + s[3*dim_2:4*dim_2]


In [4]:
def relabel_NxNxN_N_state(state):
    # Split the string into individual elements.
    state_list = state.split(';')
    
    dim = int(np.sqrt(len(state_list) // 6))
    dim_2 = dim**2
    
    if state[0] == 'N':
        for i, val in enumerate(state_list):
            # Extract the number part from the string like 'N33' -> 33.
            num = int(val[1:])
            # Replace based on the given criteria.
            if 0 <= num < dim_2:
                state_list[i] = 'A'
            elif dim_2 <= num < 2*dim_2:
                state_list[i] = 'B'
            elif 2*dim_2 <= num < 3*dim_2:
                state_list[i] = 'C'
            elif 3*dim_2 <= num < 4*dim_2:
                state_list[i] = 'D'
            elif 4*dim_2 <= num < 5*dim_2:
                state_list[i] = 'E'
            elif 5*dim_2 <= num < 6*dim_2:
                state_list[i] = 'F'
    
    return ';'.join(state_list)  

In [5]:
def apply_moves(initial_state, moves, mmoves):
    state = initial_state.split(";")
    for move_name in mmoves.split('.'):
        state = [state[i] for i in moves[move_name]]
    
    return ';'.join(state)



In [6]:
def sym_rotations(dim, mmoves, new_state, sol_state, num_wildcards, wildcard=False):
    I = ['.'.join([f'{j}{i}' for i in range(dim)]) for j in ['r', 'd', 'f']]
    manipulations = (
            [''] + 
            I + 
            [i1 + '.' + i2 for i1 in I for i2 in I] +
            [i1 + '.' + i2+ '.' + i3 for i1 in I for i2 in I for i3 in I] +
            [i1 + '.' + i2+ '.' + i3 + '.' + i4 for i1 in I for i2 in I for i3 in I for i4 in I]
    )
    
    for rotation in manipulations:
        temp_state = new_state
        if len(rotation) > 0:
            for move in rotation.split('.'):
                temp_state = ';'.join(list(np.asarray(temp_state.split(';'))[np.array(moves[move])]))
        
        if wildcard: 
            total_correct = np.sum([sol_state[i] == ns for i, ns in enumerate(temp_state.split(';'))])
            if total_correct >= 6*(dim**2) - num_wildcards:
                print(f'solved id: {id}')
                if len(rotation) > 0:
                    mmoves += '.' + rotation
                break
        else:
            if temp_state == sol_state:
                print(f'solved id: {id}')
                if len(rotation) > 0:
                    mmoves += '.' + rotation
                break
                
    return temp_state, mmoves

In [7]:
def move_translation(dim):
    M = {}
    M["U"] = f'-d{dim-1}'
    M["R"] = "r0"
    M["B"] = f"-f{dim-1}"
    M["F"] = "f0"
    M["L"] = f"-r{dim-1}"
    M["D"] = "d0"
    
    
    I = ['.'.join([f'{j}{i}' for i in range(dim)]) for j in ['r', 'f', 'd']]
    for i, j in enumerate(['x', 'y', 'z']):
        M[j] = I[i]

    if dim > 3:
        M["Uw"] = f'-d{dim-2}.-d{dim-1}'
        M["Rw"] = f"r0.r1"
        M["Bw"] = f'-f{dim-2}.-f{dim-1}'
        M["Fw"] = f"f0.f1"
        M["Lw"] = f'-r{dim-2}.-r{dim-1}'
        M["Dw"] = f"d0.d1"
        
    if dim >= 6:
        M["2Uw"] = f'-d{dim-2}.-d{dim-1}'
        M["2Rw"] = f"r0.r1"
        M["2Bw"] = f'-f{dim-2}.-f{dim-1}'
        M["2Fw"] = f"f0.f1"
        M["2Lw"] = f'-r{dim-2}.-r{dim-1}'
        M["2Dw"] = f"d0.d1"

        width_max = dim // 2
        for i in range(3, width_max + 1):
            M[f"{i}Uw"] = f'-d{dim-i}.' + M[f"{i-1}Uw"]
            M[f"{i}Rw"] = M[f"{i-1}Rw"] + f'.r{i-1}'
            M[f"{i}Bw"] = f'-f{dim-i}.' + M[f"{i-1}Bw"]
            M[f"{i}Fw"] = M[f"{i-1}Fw"] + f'.f{i-1}'
            M[f"{i}Lw"] = f'-r{dim-i}.' + M[f"{i-1}Lw"]
            M[f"{i}Dw"] = M[f"{i-1}Dw"] + f'.d{i-1}'


    for m in list(M):
        M[m+"2"] = M[m] + '.' + M[m]
        if "-" in M[m]:
            M[m+"'"] = M[m].replace("-","")
        else:
            M[m+"'"] = '.'.join(["-"+i for i in M[m].split('.')])
    
    return M

In [8]:
def get_moves(allowed_moves):
    moves = eval(allowed_moves)
    for move in list(moves):
        moves['-'+move] = np.argsort(moves[move]).tolist()
    return moves

def flip_diag_faces(M):
    # flip_diag_faces = "x' y Rw U Rw' U' F F U Rw U' Rw' F" # F"
    flip_diag_faces = "Rw U2 Rw' U2 Rw' F Rw2 U2 Rw' U2 Rw U Rw' F'"
    mm = '.'.join([M[m] for m in flip_diag_faces.split(' ')])
    return mm


def swap_faces(M):
    moves = "U F2 D Rw U2 Rw' U2 Rw' F Rw2 U2 Rw' U2 Rw U Rw' F' D' F2 U'"
    mm = '.'.join([M[m] for m in moves.split(' ')])
    return mm

        

In [9]:
def solve(initial_state, solution_state, num_wildcards, moves, verbose=False):
    init_state = relabel_NxNxN_N_state(initial_state)
    sol_state = relabel_NxNxN_N_state(solution_state)
        
    # new_moves = random.choices(list(moves.keys()), k=1)
    
    # init_state = apply_moves(init_state, moves, '.'.join(new_moves))
    # print(init_state)
    state = state2ubl(init_state)
    print(f'Starting {id}')
    output = !./rubiks-cube-solver.py --state $state
    # return output
    sol = None
    if output[-1][:9] == 'Solution:':
        # print(output[-1])
        sol = output[-1].split(': ')[1]
    else:
        for n in range(1, 21):
            if 'Solution:' in output[-n]:
                sol = output[-n].split('Solution: ')[1].split('2023-')[0]
                break
    if sol is None:
        print(output[-1])
        return output
                
    mmoves = '.'.join([M[m] for m in sol.split(' ')])
    # # new_moves_str = '.'.join(new_moves)
    # 
    # new_state = init_state
    # 
    # original_init = row['initial_state']
    # original_sol = row['solution_state'].split(';')
    # 
    # if num_wildcards >= 2:
    #     for idx, move in enumerate(mmoves.split('.')):
    #         new_state = ';'.join(list(np.asarray(new_state.split(';'))[np.array(moves[move])]))
    #         original_init = ';'.join(list(np.asarray(original_init.split(';'))[np.array(moves[move])]))
    #         original_init, mmoves = sym_rotations(dim, mmoves, original_init, original_sol, num_wildcards)
    #         total_correct = np.sum([original_sol[i] == ns for i, ns in enumerate(original_init.split(';'))])
    #         if total_correct >= 6*dim**2 - num_wildcards:
    #             print('WILDCARD SOLUTION FOUND')
    #             mmoves = '.'.join(mmoves.split('.')[:idx+1])
    #             break
    #         
    # check rotations
    
    new_state = apply_moves(initial_state, moves, mmoves)
    temp_state, mmoves = sym_rotations(dim, mmoves, new_state, solution_state.split(';'), num_wildcards, wildcard=False)

    return mmoves, sol

<a id="4"></a>
# <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:120%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" >SOLVE<br><div> 

In [10]:
# Connect to the SQLite database
conn = sqlite3.connect(database_file)
cursor = conn.cursor()

for id in range(206, 207):
    row = puzzles.loc[id]
    if row['puzzle_type'][:4] == 'cube' and row['solution_state'][0] == 'N':
        dim = int(row['puzzle_type'].split('/')[-1])
        moves = get_moves(puzzle_info.loc[row['puzzle_type'], 'allowed_moves'])
        M = move_translation(dim)
        
        # flip_moves = swap_faces(M)
        # for i in range(random.randint(0, 9)):
        #     if i % 2:
        #         flip_moves = f'{flip_moves}.r0.r1.r2.r3'
        #     elif i % 3:
        #         flip_moves = f'{flip_moves}.d0.d1.d2.d3'
        #     else:
        #         flip_moves = f'{flip_moves}.f0.f1.f2.f3'
        #     flip_moves = f'{flip_moves}.{flip_moves}'
        # state = apply_moves(row['initial_state'], moves, flip_moves)
        select_query = "SELECT moves, count FROM solutions WHERE id = ?"
    
        # Execute the query
        cursor.execute(select_query, (id,))
        response = cursor.fetchone()
        best_moves = response[0]
        best_moves_length = response[1]
        
        # use known solution to guide N-type solutions
        best_moves_list = best_moves.split('.')
        for i in range(1, len(best_moves_list)):
            if i % 10 != 0:
                continue
            known_moves = '.'.join(best_moves_list[:i])
            # known_moves = f'{known_moves}.r3.f2.-r3.f2.r3.f2.-r3.f2'
            state = apply_moves(row['initial_state'], moves, known_moves)
            # break
            
        
            init_moves, solver_moves = solve(state, row['solution_state'], row['num_wildcards'], moves, verbose=False) 
            # init_moves = solve(row['initial_state'], row['solution_state'], row['num_wildcards'], moves, verbose=False) 
            init_moves = f'{known_moves}.{init_moves}'
            # validation
            state = row['initial_state'].split(";")
            # mmoves = f'{new_moves_str}.{mmoves}'
            for move_name in init_moves.split('.'):
                state = [state[i] for i in moves[move_name]]
            
            
            # while True:
            # state = apply_moves(row['initial_state'], moves, mmoves)
            print(state)
            try:
                assert row['solution_state'].split(";") == state
                print('SOLVED with {}')
                # mmoves_length = len(mmoves.split('.'))
                
                select_query = "SELECT count FROM solutions WHERE id = ?"
    
                # Execute the query
                cursor.execute(select_query, (id,))
                response = cursor.fetchone()
                best_moves_length = response[0]
                
                # if mmoves_length < best_moves_length:
                #     # Insert the moves into the database
                #     insert_query = ("INSERT OR REPLACE INTO solutions (id, moves, count, solution_method) VALUES (?, ?, ?, ?)")
                #     cursor.execute(insert_query, (id, mmoves, mmoves_length, solution_method))
                #     conn.commit()
                break
            except AssertionError:
                print(f"assertion error for {id}")
                # print(state)
                # break
                # flip_moves = flip_diag_faces(M)
                # for _ in range(random.randint(0, 5)):
                #     flip_moves = f'{flip_moves}.{flip_moves}'
                #     
                # init_moves = f'{init_moves}.{flip_moves}'
                # state = apply_moves(';'.join(state), moves, flip_moves)
                # # for idx, move in enumerate(flip_moves.split('.')):
                # #     state = list(np.asarray(state)[np.array(moves[move])])
                # 
                # mmoves, solver_moves = solve(state, row['solution_state'], row['num_wildcards'], moves, verbose=False)
                # init_moves = f'{init_moves}.{mmoves}'
                # # validation
                # state = row['initial_state'].split(";")
                # # mmoves = f'{new_moves_str}.{mmoves}'
                # for move_name in init_moves.split('.'):
                #     state = [state[i] for i in moves[move_name]]
                # 
        
                

# Commit the changes and close the connection
conn.commit()
conn.close()


Starting 206
['N0', 'N1', 'N2', 'N3', 'N4', 'N9', 'N10', 'N7', 'N8', 'N5', 'N6', 'N11', 'N12', 'N13', 'N14', 'N15', 'N16', 'N17', 'N18', 'N19', 'N20', 'N25', 'N21', 'N23', 'N24', 'N26', 'N22', 'N27', 'N28', 'N29', 'N30', 'N31', 'N32', 'N33', 'N34', 'N35', 'N36', 'N38', 'N42', 'N39', 'N40', 'N41', 'N37', 'N43', 'N44', 'N45', 'N46', 'N47', 'N48', 'N49', 'N50', 'N51', 'N52', 'N53', 'N57', 'N55', 'N56', 'N54', 'N58', 'N59', 'N60', 'N61', 'N62', 'N63', 'N64', 'N65', 'N66', 'N67', 'N68', 'N69', 'N74', 'N71', 'N72', 'N73', 'N70', 'N75', 'N76', 'N77', 'N78', 'N79', 'N80', 'N81', 'N82', 'N83', 'N84', 'N89', 'N90', 'N87', 'N88', 'N86', 'N85', 'N91', 'N92', 'N93', 'N94', 'N95']
assertion error for 206
Starting 206
['N0', 'N1', 'N2', 'N3', 'N4', 'N9', 'N10', 'N7', 'N8', 'N6', 'N5', 'N11', 'N12', 'N13', 'N14', 'N15', 'N16', 'N17', 'N18', 'N19', 'N20', 'N25', 'N21', 'N23', 'N24', 'N26', 'N22', 'N27', 'N28', 'N29', 'N30', 'N31', 'N32', 'N33', 'N34', 'N35', 'N36', 'N38', 'N37', 'N39', 'N40', 'N41', 'N

Run in reverse?
Relabel so initial state is solution state...
Color is meaningless for N-type problem.
label initial state so it looks like a traditional solution...Some freedom to choose color positioning might be helpful
use map to color solution state... 

In [17]:
i
len(init_moves.split('.'))
# known_moves
# '.'.join(best_moves_list[514:518])

457

In [12]:
M['x'] = 'r0.r1.r2.r3'
M["x'"] = '-r0.-r1.-r2.-r3'

# edge flip
sol2 = "Rw U2 x Rw U2 Rw U2 Rw' U2 Lw U2 Rw' U2 Rw U2 Rw' U2 Rw'"

mm = '.'.join([M[m] for m in sol2.split(' ')])


mm

'r0.r1.-d3.-d3.r0.r1.r2.r3.r0.r1.-d3.-d3.r0.r1.-d3.-d3.-r0.-r1.-d3.-d3.-r2.-r3.-d3.-d3.-r0.-r1.-d3.-d3.r0.r1.-d3.-d3.-r0.-r1.-d3.-d3.-r0.-r1'

In [13]:
temp = row['initial_state'].split(';')
print(temp[0:16])
for idx, move in enumerate(init_moves.split('.')):
    temp = list(np.asarray(temp)[np.array(moves[move])])
print(temp[0:16])

['N83', 'N94', 'N82', 'N51', 'N77', 'N89', 'N54', 'N55', 'N24', 'N10', 'N74', 'N29', 'N76', 'N34', 'N1', 'N47']
['N0', 'N1', 'N2', 'N3', 'N4', 'N5', 'N6', 'N7', 'N8', 'N9', 'N10', 'N11', 'N12', 'N13', 'N14', 'N15']


In [14]:
init_moves[:-12]

'-d3.-f2.-r3.r1.-d3.-d2.r1.-f3.r3.f0.-r2.-r2.r1.r1.-f3.r3.r3.f2.f0.-d1.-f3.r3.d0.-d1.-f3.r3.d0.f0.-d1.-f2.-r3.-d3.-d2.r1.-f3.r3.f0.-r2.-d2.r1.-f3.-d3.-d2.r1.-f3.r3.f0.-r2.-d2.r1.-f3.r3.f2.-d0.-r3.f3.f0.-d1.-f3.r3.d0.-r1.f0.r2.-f1.r2.-r1.-f0.-r2.r1.r1.-f3.r3.r3.f2.-r0.f0.-d2.r1.-f3.r3.f2.r1.r1.-f3.r3.r3.f2.-d1.-f3.r3.d0.-r3.-f3.r3.d0.-d1.-f3.r3.d0.-d3.-f2.-r3.r1.-d3.-d2.r1.-f3.r3.f0.-r2.-d0.-r3.f3.d1.r2.-f1.r3.f2.d3.f3.r2.d3.-f2.r2.-d0.-r3.f3.d1.-f0.-f0.-r2.-f3.-d3.-f2.-r3.-r3.f3.-r1.-r1.r2.f0.r1.-r2.f1.-r2.-f0.r1.-r2.-f3.-d3.-f2.-r3.-f2.-r3.-d3.-d2.r1.-f3.r3.f0.-r2.-d2.r1.-f3.r3.f2.-d3.-f2.-r3.r1.f1.-r2.-f0.r1.-d3.-d2.r1.-f3.r3.f0.-r2.-d2.r1.-f3.r3.f2.d3.-f2.r1.r1.-f3.r3.r3.f2.r2.f0.r1.-r2.-d3.-d2.r1.-f3.r3.f0.-r2.-d2.r1.-f3.r3.f0.-r2.-f0.r2.-r0.f0.r1.-f3.r3.-f0.-r2.-f3.-d3.-f2.-r3.r1.f0.-r0.f0.r1.-f3.r3.f0.r1.-r2.r1.-f1.-r0.f0.-r0.f0.-r3.r1.-f3.r3.-f3.-d3.-d0.-r3.f3.-d1.-f3.r3.d0.-d2.r1.-f3.r3.r2.-r0.f0.r1.-f3.r3.-f2.-r3.-f0.r1.r1.-f3.r3.r3.f2.-d2.r1.-f3.r3.f2.-r0.f0.-d2.r1.-f3.r3.f2.

In [15]:
temp = state.copy()
dim_2 = 4**2
face = 0
print(state[dim_2 * face + 5:dim_2 * face + 7])
print(state[dim_2 * face + 9:dim_2 * face + 11])
for idx, move in enumerate(mm.split('.')):
    temp = list(np.asarray(temp)[np.array(moves[move])])
print(temp[dim_2 * face + 5:dim_2 * face + 7])
print(temp[dim_2 * face + 9:dim_2 * face + 11])

['N5', 'N6']
['N9', 'N10']
['N10', 'N9']
['N6', 'N5']


In [16]:
print(temp)

['N0', 'N1', 'N2', 'N3', 'N11', 'N10', 'N9', 'N8', 'N7', 'N6', 'N5', 'N4', 'N19', 'N18', 'N17', 'N16', 'N15', 'N14', 'N13', 'N12', 'N20', 'N21', 'N22', 'N23', 'N24', 'N25', 'N26', 'N27', 'N28', 'N29', 'N30', 'N31', 'N67', 'N65', 'N66', 'N35', 'N36', 'N41', 'N37', 'N39', 'N40', 'N42', 'N38', 'N43', 'N44', 'N45', 'N46', 'N47', 'N48', 'N49', 'N50', 'N51', 'N52', 'N53', 'N54', 'N55', 'N56', 'N57', 'N58', 'N59', 'N60', 'N61', 'N62', 'N63', 'N64', 'N33', 'N34', 'N32', 'N68', 'N69', 'N70', 'N71', 'N72', 'N73', 'N74', 'N75', 'N76', 'N77', 'N78', 'N79', 'N80', 'N81', 'N82', 'N83', 'N84', 'N85', 'N86', 'N87', 'N88', 'N89', 'N90', 'N91', 'N92', 'N93', 'N94', 'N95']
